# Logic Apps Connectors as MCP Tools

> **Author:** Ozgur Guler | AI Solution Leader, AI Innovation Hub
> **Contact:** [ozgur.guler1@gmail.com](mailto:ozgur.guler1@gmail.com)
> **© 2025 Ozgur Guler. All rights reserved.**

---

Use 1,400+ Logic Apps connectors (Salesforce, ServiceNow, SAP, etc.) as AI agent tools.

## Supported Connectors

| Connector | Use Cases |
|-----------|----------|
| **Salesforce** | Query contacts, create opportunities, update accounts |
| **ServiceNow** | Create incidents, update tickets, assign tasks |
| **SAP** | Read business data, trigger workflows |
| **Dynamics 365** | CRM operations, sales pipeline |
| **SQL/Dataverse** | Database queries, record updates |
| **GitHub** | Issues, PRs, repository management |

---

## Setup: Expose Connector as MCP Tool

### Via Foundry Portal

1. Go to **Agent Tools** catalog
2. Search for connector (e.g., "Salesforce")
3. Select **Logic App resource** (or create new)
4. Choose **operations** to expose
5. Configure **parameters** (AI-provided vs user-provided)
6. **Register** in Foundry

### MCP Endpoint Format

```
https://<logic-app>.azurewebsites.net/api/mcpservers/<connector-name>/mcp
```

In [ ]:
!pip install azure-ai-projects azure-ai-agents azure-identity --pre --quiet

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv("../.env")

PROJECT_ENDPOINT = os.getenv(
    "PROJECT_ENDPOINT",
    "https://ozgurguler-7212-resource.services.ai.azure.com/api/projects/ozgurguler-7212"
)

# Example: ServiceNow connector exposed as MCP
SERVICENOW_MCP_URL = os.getenv(
    "SERVICENOW_MCP_URL",
    "https://your-logic-app.azurewebsites.net/api/mcpservers/servicenow/mcp"
)

# Example: Salesforce connector exposed as MCP  
SALESFORCE_MCP_URL = os.getenv(
    "SALESFORCE_MCP_URL",
    "https://your-logic-app.azurewebsites.net/api/mcpservers/salesforce/mcp"
)

MODEL = os.getenv("MODEL_DEPLOYMENT_NAME", "gpt-5-nano")

---

## Example 1: ServiceNow Incident Management

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import McpTool

client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=DefaultAzureCredential())

# ServiceNow connector as MCP tool
servicenow_tool = McpTool(
    server_label="servicenow",
    server_url=SERVICENOW_MCP_URL,
    allowed_tools=["CreateIncident", "UpdateIncident", "GetIncident"],
)

# Create IT support agent
agent = client.agents.create_agent(
    model=MODEL,
    name="it-support-agent",
    instructions="""You are an IT support agent. Use ServiceNow to:
- Create incidents for reported issues
- Update ticket status
- Look up existing incidents""",
    tools=servicenow_tool.definitions,
)

print(f"Created agent: {agent.id}")

In [ ]:
import time
from azure.ai.agents.models import ListSortOrder, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval

def run_agent(agent, client, mcp_tool, user_message):
    """Run agent with MCP tool and return response."""
    thread = client.agents.threads.create()
    client.agents.messages.create(thread_id=thread.id, role="user", content=user_message)
    
    run = client.agents.runs.create(
        thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources
    )
    
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = client.agents.runs.get(thread_id=thread.id, run_id=run.id)
        
        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            approvals = [
                ToolApproval(tool_call_id=tc.id, approve=True, headers=mcp_tool.headers)
                for tc in run.required_action.submit_tool_approval.tool_calls
                if isinstance(tc, RequiredMcpToolCall)
            ]
            if approvals:
                client.agents.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_approvals=approvals)
    
    messages = client.agents.messages.list(thread_id=thread.id, order=ListSortOrder.DESCENDING)
    for msg in messages:
        if msg.role == "assistant" and msg.text_messages:
            return msg.text_messages[-1].text.value
    return None

# Test ServiceNow agent
response = run_agent(
    agent, client, servicenow_tool,
    "Create a P2 incident: User unable to access email on mobile device"
)
print(f"Agent: {response}")

In [ ]:
client.agents.delete_agent(agent.id)
print("ServiceNow agent deleted")

---

## Example 2: Salesforce CRM Operations

In [ ]:
# Salesforce connector as MCP tool
salesforce_tool = McpTool(
    server_label="salesforce",
    server_url=SALESFORCE_MCP_URL,
    allowed_tools=["GetContact", "CreateOpportunity", "UpdateAccount"],
)

# Create sales assistant agent
sales_agent = client.agents.create_agent(
    model=MODEL,
    name="sales-assistant-agent",
    instructions="""You are a sales assistant. Use Salesforce to:
- Look up customer contacts
- Create new opportunities
- Update account information""",
    tools=salesforce_tool.definitions,
)

print(f"Created sales agent: {sales_agent.id}")

In [ ]:
# Test Salesforce agent
response = run_agent(
    sales_agent, client, salesforce_tool,
    "Look up contact info for Acme Corporation"
)
print(f"Agent: {response}")

In [ ]:
client.agents.delete_agent(sales_agent.id)
print("Sales agent deleted")

---

## Example 3: Multi-Connector Agent

Combine multiple connectors in one agent.

In [ ]:
# Multiple MCP tools for one agent
servicenow_tool = McpTool(server_label="servicenow", server_url=SERVICENOW_MCP_URL)
salesforce_tool = McpTool(server_label="salesforce", server_url=SALESFORCE_MCP_URL)

# Combine tool definitions
all_tools = servicenow_tool.definitions + salesforce_tool.definitions

# Create multi-connector agent
multi_agent = client.agents.create_agent(
    model=MODEL,
    name="enterprise-assistant",
    instructions="""You are an enterprise assistant with access to:
- ServiceNow for IT incidents
- Salesforce for CRM operations
Route requests to the appropriate system.""",
    tools=all_tools,
)

print(f"Created multi-connector agent: {multi_agent.id}")

# Cleanup
client.agents.delete_agent(multi_agent.id)

---

## Connector Setup Reference

### Workflow Requirements

Each connector operation needs a workflow with:
1. **Request trigger** - HTTP endpoint
2. **Connector action** - The actual API call
3. **Response action** - Return result

### Example: ServiceNow CreateIncident Workflow

```json
{
  "definition": {
    "triggers": {
      "manual": { "type": "Request", "kind": "Http" }
    },
    "actions": {
      "Create_Record": {
        "type": "ApiConnection",
        "inputs": {
          "host": { "connection": { "name": "@parameters('$connections')['servicenow']['connectionId']" } },
          "method": "post",
          "path": "/api/now/v2/table/incident",
          "body": {
            "short_description": "@triggerBody()?['description']",
            "priority": "@triggerBody()?['priority']"
          }
        }
      },
      "Response": {
        "type": "Response",
        "inputs": { "statusCode": 200, "body": "@body('Create_Record')" }
      }
    }
  }
}
```

---

## Summary

| Connector | MCP Tool | Agent Use Case |
|-----------|----------|----------------|
| ServiceNow | `McpTool(server_label="servicenow")` | IT incident management |
| Salesforce | `McpTool(server_label="salesforce")` | CRM operations |
| SAP | `McpTool(server_label="sap")` | ERP data access |
| Dynamics 365 | `McpTool(server_label="dynamics")` | Business apps |

## Next

Continue to `../09-apim-ai-gateway-mcp`.

---

<div align="center">

## License & Attribution

This notebook is part of the **Azure AI Foundry Demo Repository**

[![License](https://img.shields.io/badge/License-Apache%202.0-blue.svg)](../LICENSE)

**Original Author:** Ozgur Guler | AI Solution Leader, AI Innovation Hub

**Contact:** [ozgur.guler1@gmail.com](mailto:ozgur.guler1@gmail.com)

---

*If you use, modify, or distribute this work, you must provide appropriate credit to the original author as required by the [Apache License 2.0](../LICENSE).*

**Copyright © 2025 Ozgur Guler. All rights reserved.**

</div>